<a href="https://colab.research.google.com/github/udsey/SATO_RL/blob/main/DQN27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Import

In [51]:
pip uninstall jedi --yes

Found existing installation: jedi 0.17.1
Uninstalling jedi-0.17.1:
  Successfully uninstalled jedi-0.17.1
Note: you may need to restart the kernel to use updated packages.


In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import plotly.graph_objects as go
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, InputLayer
from keras.optimizers import Adam
from collections import deque

In [2]:
import plotly.io as pio
pio.renderers.default = 'notebook'

# Functions

In [3]:
def limit_plot(df, return_df = False, plot=True):
    import plotly.graph_objects as go
    df_lim = df.copy()
    df_lim['x'] = df_lim.apply(lambda x: np.arange(x[0], x[1], 0.1), axis=1)
    df_lim = df_lim.explode('x')
    df_lim.drop(columns=df_lim.columns[0:2], inplace=True)
    df_lim.rename(columns={df_lim.columns[0]:'y'}, inplace=True)
    df_lim['y'] *= 3600/1000
    if plot == True:
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=df_lim.x, y=df_lim.y, name='speed limit'))
        fig.show()
    if return_df == True:
        return df_lim
def plot_stat(hist, total_reward_list):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(hist)
    plt.title('Iteration number') 
    plt.subplot(1, 2, 2)
    plt.plot(total_reward_list)
    plt.title('Total reward')      

def plot_result(df, df_limit_speed):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_limit_speed.x, y=df_limit_speed.y, name='speed limit'))
    fig.add_trace(go.Scatter(x = df['position_m'], y = df['speed_km/h'], name='train movement'))
    #fig.show('svg')
    fig.show()

def min_time(s, v, a=0.5):
    s*=1000
    v = v / 3.6
    t = v/a
    s_0 = a*t**2/2
    s_ost = s-s_0
    t_1 = s_ost/v
    t_min  = t+t_1
    return t_min

def ideal_driving(v_limit, s, t, dt):
    env = Train(t, v_limit, s, dt)
    done = False
    state = env.reset()
    v_limit = v_limit*1000/3600
    v = 0
    sum_reward = 0
    action = 0
    pos_action = env.action_high
    state_list = []
    action_list = []
    state_list.append(state)
    while not done:
        v += action * dt
        next_v = v + env.action_high * dt
        #next_v = v
        if next_v <= v_limit:
            action = env.action_high
        elif v == v_limit:
            action = 0
        else:
            action = (v_limit - next_v)/dt
        state, reward, done = env.step(action)
        state_list.append(state)
        action_list.append(action)
        sum_reward += reward
    df = env.legend()
    df_limit_speed = limit_plot(env.df_limit, return_df=True, plot=False)
    print('Total reward = ', sum_reward)
    plot_result(df, df_limit_speed)
    state_list = state_list[:-1]
    return df, state_list, action_list

def get_true_action(state):
    s, v, v_limit = state
    dt = env.dt
    next_v = v + env.action_high * dt
    if next_v <= v_limit:
        action = env.action_high
    elif v == v_limit:
        action = 0
    else:
        action = (v_limit - next_v)/dt
    return np.array([action])

# Train model

In [12]:
class Train:


    def __init__(
                 self, 
                 time_limit=2000, # (sec), available time for the route;
                 v_limit=100, # (km/h), speed limit on the route;
                 s_limit=300, # (km), distance;
                 dt=10, # (sec), state-changing frequency;
                 ):

        """

        Init train

        Parameters:
        time_limit - (sec), available time for the route;
        v_limit - (km/h), speed limit on the route;
        s_limit - (km), distance;
        dt - (sec), state-changing frequency;

        """

        self.action_dim = 1
        self.observation_space = 3

        self.action_high = 0.5 # m/s**2
        self.action_low = -0.5 # m/s**2
        self.action_space = np.array(np.arange(self.action_low, self.action_high, 0.1))
        self.time_limit = time_limit # s
        self.v_limit = v_limit * 1000/3600 # m/s
        self.s_limit = s_limit * 1000 # m
        self.dt = dt # s

        self.df_limit = pd.DataFrame({'start': [0],
                                      'stop': [self.s_limit],
                                      'limit': self.v_limit})
        self.reset()

    def append_velocity_limit(self, limit, start, stop):
        """

        Adds speed limit

        Parameters:
        limit - (km/h), speed limit on the road section;
        start - (km), start point of the current speed limit;
        stop - (km), end point of the current speed limit;

        Return:

        pd.DataFrame with columns start (m), stop (m), limit (m/s)

        """
        self.df_limit = self.df_limit.append({'start': start * 1000,
                                              'stop': stop * 1000,
                                              'limit': limit * 1000/3600},
                                              ignore_index=True)
        return self.df_limit

    def reset_velocity_limit(self):
        """
        Reset all speed limits 
        """
        self.append_velocity_limit(self, limit=self.v_limit, 
                                   position=range(0, self.s_limit))

    def reset(self):
        """

        Reset environment to initial state

        """
        self.s = 0
        self.v = 0
        self.a = 0
        self.done = False
        self.total_time = 0
        self.reward = 0
        self.v_list = []
        self.s_list = []
        self.t_list = []
        self.a_list = []
        self.reward_list = []
        self.done_list = []

        return self.return_state()
    def return_state(self):
        """

        Return current state, 
        a list with structure:[position(m), velocity(m/s), velocity_limit(m/s)]

        """
        return [np.float(self.s), np.float(self.v), np.float(self.speed_limit())]


    def speed_limit(self):
        """

        Return the speed limit at the current point

        """
        if self.s < 0:
            return self.df_limit.iloc[0, 2]
    
        for i in range(self.df_limit.shape[0]):
            min_p = self.df_limit.iloc[i, 0]
            max_p = self.df_limit.iloc[i, 1]
            if min_p <= self.s <= max_p:
                return self.df_limit.iloc[i, 2]
            else:
                return min_p

    def legend(self):

        """
        Return pd.DataFrame with movement history

        """
        df = pd.DataFrame(columns=['time_s', 'position_m', 'speed_m/s', 'speed_km/h', 'acceleration_m/s^2', 'reward', 'done'])
        df['time_s'] = self.t_list
        df['position_m'] = self.s_list
        df['speed_m/s'] = self.v_list
        df['speed_km/h'] = np.round(np.array(self.v_list) * 3600/1000 , 0)
        df['acceleration_m/s^2'] = self.a_list
        df['reward'] = self.reward_list
        df['done'] = self.done_list

        return df


    def add_to_hist_df(self, velocity, time, position,
                       acceleration, reward, done):

        self.v_list.append(velocity)
        self.t_list.append(time)
        self.s_list.append(position)
        self.a_list.append(acceleration)
        self.reward_list.append(reward)
        self.done_list.append(done)


    def reward_func(self):
        """

        Return reward at the current point and flag if the action is done

        """
        
        if self.s < 0 or self.v < 0: # had driven the other way
            return -1000, True
        if self.s >= self.s_limit: # attained destination
            self.s = np.array(self.s_limit)
            return 100, True
        if self.total_time > self.time_limit: # exceeded time limit
            return -100, True
        if self.v > self.speed_limit(): # exceeded speed limit
            return -500, True
        if self.v <= self.speed_limit(): # moves at an acceptable speed
            return 1, False
        
       
    def step(self, action):

        """
        Step for the environment

        Parameters:
        action - acceleration (m/s^2)

        Return:
        State[], Reward, Done
        """

        
        self.total_time += self.dt # (s) - total operating time
        self.a = action # (m/s^2) - acceleration
        self.s += self.v * self.dt + (self.a * (self.dt ** 2))/2 # (m) - current position
        self.v += (self.a * self.dt) # (m/s) - current speed
        self.reward, self.done = self.reward_func()

        self.add_to_hist_df(self.v, self.total_time,
                       self.s, self.a, self.reward, self.done)
        
        return  self.return_state(), self.reward, self.done

# DQN

In [13]:
class DQN:
    def __init__(self, env, memory_size=2000, batch_size=64,
                 gamma=0.85, tau=0.125, learning_rate=0.005,
                 epsilon=1, epsilon_min=0.01, epsilon_decay=0.995):
        
        self.env = env
        self.memory = deque(maxlen=memory_size)
        
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.learning_rate = learning_rate
        self.tau = tau
        self.batch_size = batch_size

        self.model = self.create_model()
        self.target_model = self.create_model()

    def create_model(self):

        optimizer = Adam(learning_rate=self.learning_rate)
        model = Sequential()
        state_shape  = 3
        model.add(InputLayer(input_shape=(state_shape,)))
        model.add(Dense(24, activation="relu"))
        model.add(Dense(48, activation="relu"))
        model.add(Dense(24, activation="relu"))
        model.add(Dense(self.env.action_space.shape[0]))
        model.compile(loss="mean_squared_error", optimizer=optimizer)
        return model

    def act(self, state):
        self.epsilon *= self.epsilon_decay
        self.epsilon = max(self.epsilon_min, self.epsilon)
        if np.random.random() < self.epsilon:
            return np.random.randint(env.action_space.shape)
        else:
            state = np.expand_dims(state, 0)
            return np.argmax(self.model.predict(state)[0])

    def remember(self, state, action, reward, new_state, done):
        self.memory.append([state, action, reward, new_state, done])

    def replay(self):
        batch_size = self.batch_size
        if len(self.memory) < batch_size: 
            return

        samples = random.sample(self.memory, batch_size)
        for sample in samples:
            state, action, reward, new_state, done = sample
            state = np.expand_dims(state, 0)
            new_state = np.expand_dims(new_state, 0)
            target = self.target_model.predict(state)
            if done:
                target[0][action] = reward
            else:
                Q_future = max(self.target_model.predict(new_state)[0])
                target[0][action] = reward + Q_future * self.gamma
            self.model.fit(state, target, epochs=1, verbose=0)

    def target_train(self):
        weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = weights[i] * self.tau + target_weights[i] * (1 - self.tau)
        self.target_model.set_weights(target_weights)

    def save_model(self, fn):
        self.model.save(fn)

# Fit/Predict

In [14]:
s = 2
v = 100
dt = 1
t = min_time(s, v)*1.1

In [15]:
df, _, _ = ideal_driving(v, s, t, dt)

Total reward =  199


In [16]:
env = Train(t, v, s, dt)
df_limit_speed = limit_plot(env.df_limit, return_df=True)

In [ ]:
env = Train(t, v, s, dt)
gamma   = 0.9
epsilon = .95
trials  = 1000
trial_len = 500

    # updateTargetNetwork = 1000
dqn_agent = DQN(env)
steps = []
reward_list = []
for trial in range(trials):
    cur_state = env.reset()
    step = 0
    total_reward = 0
    done = False
    while not done:

        step += 1
        action = dqn_agent.act(cur_state)
        new_state, reward, done = env.step(action)

            # reward = reward if not done else -20
        new_state = new_state
        dqn_agent.remember(cur_state, action, reward, new_state, done)
            
        dqn_agent.replay()       # internally iterates default (prediction) model
        dqn_agent.target_train() # iterates target model
        total_reward += reward
        cur_state = new_state
    steps.append(step)
    reward_list.append(total_reward)
    if trial % 10 == 0 or trial == trials-1:
        print("Episode * {} * Avg Reward is ==> {} Max reward ==> {}".format(trial,
                                                                             np.mean(reward_list[-10:]),
                                                                             max(reward_list[-10:])))
dqn_agent.save_model("success.model")
plot_stat(steps, reward_list)

Episode * 0 * Avg Reward is ==> -496.0 Max reward ==> -496
Episode * 10 * Avg Reward is ==> -494.2 Max reward ==> -492
Episode * 20 * Avg Reward is ==> -495.3 Max reward ==> -494
Episode * 30 * Avg Reward is ==> -492.3 Max reward ==> -484
Episode * 40 * Avg Reward is ==> -489.0 Max reward ==> -478
Episode * 50 * Avg Reward is ==> -487.5 Max reward ==> -474
Episode * 60 * Avg Reward is ==> -232.6 Max reward ==> 9
Episode * 70 * Avg Reward is ==> -122.4 Max reward ==> 9
Episode * 80 * Avg Reward is ==> -82.2 Max reward ==> 203
Episode * 90 * Avg Reward is ==> -116.9 Max reward ==> 9
Episode * 100 * Avg Reward is ==> 9.0 Max reward ==> 9
Episode * 110 * Avg Reward is ==> 59.1 Max reward ==> 209
Episode * 120 * Avg Reward is ==> 28.8 Max reward ==> 207
Episode * 130 * Avg Reward is ==> 3.1 Max reward ==> 209
Episode * 140 * Avg Reward is ==> 48.7 Max reward ==> 209
Episode * 150 * Avg Reward is ==> -100.1 Max reward ==> 202
Episode * 160 * Avg Reward is ==> -31.3 Max reward ==> 9
Episode *